# Regression Training

In [1]:
save_models_path = "/home/adrian_alvarez/Projects/NBA_over_under_predictor/models/tab_pfn_regressor/"
prefix_name = "regressor_tab_pfn"

## Libraries

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
from nba_ou.data_preparation.missing_data.handle_missing_data import apply_missing_policy, compute_and_save_train_medians

## Load Data

In [4]:
data_path = "/home/adrian_alvarez/Projects/NBA_over_under_predictor/data/train_data/"
name = "training_data_from_2004-10-01_to_2026-01-10.csv"
path = "/home/panchojasen/Projects/NBA_over_under_predictor/data/training_data_from_2004-10-01_to_2026-01-10.csv"

df_stats = pd.read_csv(path)

dtype_dict = {col: str for col in df_stats.columns if "ID" in col.upper()}

df_stats = pd.read_csv(
    path,
    dtype=dtype_dict
)
df_stats['GAME_DATE'] = pd.to_datetime(df_stats['GAME_DATE']).dt.strftime('%Y-%m-%d')

/tmp/ipykernel_237463/2265647666.py:5: DtypeWarning: Columns (205,206,207,248,249,250,291,292,293,334,335,336,377,378,379,420,421,422,616,617,618,659,660,661,702,703,704,745,746,747,788,789,790,831,832,833) have mixed types. Specify dtype option on import or set low_memory=False.
  df_stats = pd.read_csv(path)
/tmp/ipykernel_237463/2265647666.py:9: DtypeWarning: Columns (205,206,207,248,249,250,291,292,293,334,335,336,377,378,379,420,421,422,616,617,618,659,660,661,702,703,704,745,746,747,788,789,790,831,832,833) have mixed types. Specify dtype option on import or set low_memory=False.
  df_stats = pd.read_csv(


In [5]:
df_stats= df_stats[df_stats['TOTAL_POINTS'] > 130]

In [6]:
df_stats.shape[0]

23691

In [7]:
df_stats

,TOTAL_OVER_UNDER_LINE,TEAM_ID_TEAM_HOME,TEAM_CITY_TEAM_HOME,TEAM_ABBREVIATION_TEAM_HOME,TEAM_NAME_TEAM_HOME,MATCHUP_TEAM_HOME,GAME_NUMBER_TEAM_HOME,TEAM_ID_TEAM_AWAY,TEAM_CITY_TEAM_AWAY,TEAM_ABBREVIATION_TEAM_AWAY,...,PTS_TREND_SLOPE_DIFF_HOME_MINUS_AWAY,PTS_TREND_SLOPE_SUM_HOME_PLUS_AWAY,TRAVEL_RECENCY_RATIO_HOME_2D_OVER_14D,TRAVEL_RECENCY_RATIO_AWAY_2D_OVER_14D,REST_DAYS_DIFF_HOME_MINUS_AWAY,INJURY_PTS_SHARE_HOME,INJURY_PTS_SHARE_AWAY,STAR_PTS_PCT_DIFF_HOME_MINUS_AWAY,POSS_X_TSPCT_HOME,POSS_X_TSPCT_AWAY
0,240.9999,1610612762,Utah,UTA,Utah Jazz,UTA vs. CHA,38,1610612766,Charlotte,CHA,...,1.7,-3.1,0.788795,0.000000,0,0.240990,0.133469,0.022339,60.680317,57.657276
1,240.4999,1610612739,Cleveland,CLE,Cleveland Cavaliers,CLE vs. MIN,40,1610612750,Minnesota,MIN,...,-10.7,-6.5,0.773813,0.000000,0,0.098350,0.080271,0.014506,59.123238,61.144985
2,214.4999,1610612765,Detroit,DET,Detroit Pistons,DET vs. LAC,38,1610612746,LA,LAC,...,-8.5,-3.3,0.000000,0.234741,2,0.495857,0.234389,-0.156275,58.487861,56.322250
3,236.4999,1610612754,Indiana,IND,Indiana Pacers,IND vs. MIA,39,1610612748,Miami,MIA,...,8.6,-1.6,0.727915,0.000000,-2,0.379738,0.021147,0.006671,56.971687,59.561623
4,233.4999,1610612741,Chicago,CHI,Chicago Bulls,CHI vs. DAL,38,1610612742,Dallas,DAL,...,2.3,0.1,0.000000,0.766714,1,0.289831,0.425103,-0.010188,58.130139,57.252427
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23686,215.5000,1610612740,New Orleans,NOH,New Orleans Pelicans,NOH vs. SAC,1,1610612758,Sacramento,SAC,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
23687,191.0000,1610612739,Cleveland,CLE,Cleveland Cavaliers,CLE vs. DAL,1,1610612742,Dallas,DAL,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
23688,183.5000,1610612747,Los Angeles,LAL,Los Angeles Lakers,LAL vs. HOU,1,1610612745,Houston,HOU,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
23689,208.5000,1610612759,San Antonio,SAS,San Antonio Spurs,SAS vs. POR,1,1610612757,Portland,POR,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN


In [8]:
#count nas in TOTA_OVER_UNDER_LINE
nans = df_stats['TOTAL_OVER_UNDER_LINE'].isna().sum()
print(f"Number of NaNs in TOTAL_OVER_UNDER_LINE: {nans}")

Number of NaNs in TOTAL_OVER_UNDER_LINE: 1702


In [9]:
df_stats.dropna(subset=['TOTAL_OVER_UNDER_LINE'], inplace=True)
df_stats = df_stats[df_stats['TOTAL_OVER_UNDER_LINE'] > 100]

In [10]:
df_stats['SEASON_YEAR'].unique()

array([2025, 2024, 2023, 2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015,
       2014, 2013, 2012, 2011, 2010, 2009, 2008, 2007])

In [11]:
#drop season 2007
df_stats = df_stats[df_stats['SEASON_YEAR'] != 2007]

In [12]:
df_to_train = df_stats.copy()

In [13]:
text_columns = df_to_train.select_dtypes(include=['object', 'string']).columns

# Print the names of the columns containing text
# print("Text columns:", list(text_columns))
cols_to_drop = ['SEASON_ID', 'GAME_ID', 'SEASON_TYPE',  'TEAM_ID_TEAM_HOME',
       'TEAM_ID_TEAM_AWAY','IS_OVERTIME']

cols_to_drop += [
    "MATCHUP_TEAM_HOME",
    "GAME_NUMBER_TEAM_HOME",
    "TEAM_CITY_TEAM_AWAY",
    "TEAM_ABBREVIATION_TEAM_AWAY",
    "TEAM_ABBREVIATION_TEAM_HOME",
    "TEAM_CITY_TEAM_HOME",
    "TEAM_NAME_TEAM_AWAY",
    "MATCHUP_TEAM_AWAY",
    'TEAM_NAME_TEAM_HOME'
]
cols_to_drop += list(text_columns)

cols_to_drop.pop(cols_to_drop.index('GAME_DATE'))

df_to_train.drop(columns=cols_to_drop, inplace=True, errors='ignore')


In [14]:
#Calculate number fo rows with at least 1 NA in total and in percentage
rows_with_na = df_to_train.isna().any(axis=1).sum()
total_rows = df_to_train.shape[0]
percentage_with_na = (rows_with_na / total_rows) * 100
print(f"Rows with at least one NA: {rows_with_na} out of {total_rows} ({percentage_with_na:.2f}%)")

Rows with at least one NA: 1862 out of 20775 (8.96%)


In [15]:
# Count NAs per column
na_counts = df_to_train.isna().sum()

# Get most common SEASON_YEAR for nulls in each column
most_common_season = []
for col in df_to_train.columns:
    if na_counts[col] > 0:
        # Get rows where this column is null
        null_rows = df_stats[df_stats[col].isna()]
        if len(null_rows) > 0 and 'SEASON_YEAR' in df_stats.columns:
            # Find most common SEASON_YEAR for these null rows
            common_season = null_rows['SEASON_YEAR'].mode()
            most_common_season.append(common_season.iloc[0] if len(common_season) > 0 else None)
        else:
            most_common_season.append(None)
    else:
        most_common_season.append(None)

na_counts_df = pd.DataFrame({
    'Column': na_counts.index,
    'NA_Count': na_counts.values,
    'NA_Percentage': (na_counts.values / len(df_to_train) * 100).round(2),
    'Most_Common_Season_Year': most_common_season
}).sort_values('NA_Count', ascending=False)

# Show only columns with NAs
na_counts_df[na_counts_df['NA_Count'] > 0]

,Column,NA_Count,NA_Percentage,Most_Common_Season_Year
514,TOTAL_OVER_UNDER_LINE_COMBINED_TREND_SLOPE_LAS...,627,3.02,2022.0
609,TRAVEL_RECENCY_RATIO_AWAY_2D_OVER_14D,561,2.70,2019.0
605,PTS_FORM_Z_AWAY_LAST5_VS_SEASON,561,2.70,2010.0
604,PTS_FORM_Z_HOME_LAST5_VS_SEASON,554,2.67,2022.0
494,AVG_POINTS_CONCEDED_AWAY_BEFORE_GAME,540,2.60,2008.0
...,...,...,...,...
528,REF_TRIO_DIFFERENCE_FROM_LINE_DIFF_BEFORE,23,0.11,2023.0
531,MONEYLINE_TEAM_HOME,12,0.06,2011.0
530,SPREAD,12,0.06,2011.0
597,IMPLIED_PTS_HOME,12,0.06,2011.0


In [16]:
# df_to_train.fillna(0, inplace=True)

In [17]:
df_to_train['TOTAL_OVER_UNDER_LINE'].unique()

array([240.9999, 240.4999, 214.4999, 236.4999, 233.4999, 230.4999,
       231.4999, 228.9999, 221.9999, 228.4999, 229.4999, 243.4999,
       226.9999, 223.9999, 232.4999, 238.9999, 227.4999, 232.9999,
       246.4999, 222.9999, 242.9999, 222.4999, 220.9999, 234.4999,
       237.4999, 234.9999, 237.9999, 244.4999, 236.9999, 241.9999,
       239.4999, 238.4999, 224.4999, 247.9999, 239.9999, 248.4999,
       219.9999, 235.4999, 226.4999, 216.9999, 229.9999, 249.4999,
       235.9999, 218.4999, 246.9999, 241.4999, 233.9999, 243.9999,
       231.9999, 250.4999, 225.9999, 245.4999, 254.4999, 220.4999,
       225.4999, 251.4999, 215.9999, 221.4999, 242.4999, 215.4999,
       224.9999, 223.4999, 219.4999, 244.9999, 227.9999, 248.9999,
       249.9999, 218.9999, 230.9999, 216.4999, 217.4999, 213.9999,
       210.9999, 207.9999, 203.4999, 209.4999, 201.4999, 208.9999,
       209.9999, 212.9999, 205.9999, 206.4999, 202.4999, 210.4999,
       199.4999, 211.4999, 198.9999, 211.9999, 205.4999, 200.4

In [18]:
print("Applying missing data policy...")
print(f"Data shape before handling missing data: {df_to_train.shape}")

Applying missing data policy...
Data shape before handling missing data: (20775, 616)


In [19]:
    # Training time
train_medians = compute_and_save_train_medians(df_to_train)

# Prediction time
df_to_train, report = apply_missing_policy(
    df_to_train,
    train_medians=train_medians,
    current_total_line_col="TOTAL_OVER_UNDER_LINE",
    drop_mode="strict",)

In [20]:
# report

In [21]:
print(f"Data shape after handling missing data: {df_to_train.shape}")

Data shape after handling missing data: (20379, 616)


In [22]:
null_rows = df_to_train[df_to_train.isnull().any(axis=1)]
print(f"Total number of null values in the DataFrame: {null_rows.shape[0]}")
print(f"Percentage of null values in the DataFrame: {null_rows.shape[0] / df_to_train.shape[0] * 100:.2f}%")
print(f"Total number of rows in the DataFrame: {df_to_train.shape[0]}")

null_columns = df_to_train.columns[df_to_train.isnull().any()].tolist()
print(f"Columns with null values: {null_columns}")


Total number of null values in the DataFrame: 0
Percentage of null values in the DataFrame: 0.00%
Total number of rows in the DataFrame: 20379
Columns with null values: []


In [23]:
df_to_train.drop(columns='GAME_DATE', inplace=True)

## Train / Test

In [24]:
from sklearn.model_selection import train_test_split

In [25]:
X = df_to_train.drop(cols_to_drop + ['TOTAL_POINTS'], axis=1, errors='ignore')
y = df_to_train['TOTAL_POINTS']

In [26]:
# Train / Test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=16)

In [27]:
df_to_train['IS_TRAINING_DATA'] = False

# Mark True for the rows in the training set
df_to_train.loc[X_train.index, 'IS_TRAINING_DATA'] = True
output_name = f"{data_path}/training_data_with_missing_data_handled_from_2004-10-01_to_2026-01-10.csv"
# df_to_train.to_csv(output_name, index=False)

In [28]:
print(f"Training set size: {X_train.shape[0]}")
print(f"Test set size: {X_test.shape[0]}")
# Check number of coulmns
print(f"Number of columns in training set: {X_train.shape[1]}")
print(f"Number of columns in test set: {X_test.shape[1]}")

Training set size: 10189
Test set size: 10190
Number of columns in training set: 614
Number of columns in test set: 614


In [29]:
# Custom scorer for over/under betting accuracy
def over_under_betting_accuracy(y_true, y_pred, betting_line):
    """
    Calculate the accuracy of over/under betting decisions.
    
    A bet is successful if:
    - We predict OVER (pred > line) and actual is OVER (true > line), OR
    - We predict UNDER (pred < line) and actual is UNDER (true < line)
    
    Parameters:
    -----------
    y_true : array-like
        Actual total points
    y_pred : array-like
        Predicted total points
    betting_line : array-like
        Over/under betting line
    
    Returns:
    --------
    float : Accuracy of betting decisions (0 to 1)
    """
    # Calculate differences from the betting line
    pred_diff = y_pred - betting_line
    true_diff = y_true - betting_line
    
    # Check if both have the same sign (both positive or both negative)
    # np.sign returns -1, 0, or 1
    correct_predictions = np.sign(pred_diff) == np.sign(true_diff)
    
    # Calculate accuracy
    accuracy = np.mean(correct_predictions)
    
    return accuracy


# Custom scorer class to work with sklearn
class OverUnderScorer:
    """
    Custom scorer that calculates betting accuracy for over/under predictions.
    """
    def __call__(self, estimator, X, y_true):
        """
        Calculate the over/under betting accuracy.
        
        Parameters:
        -----------
        estimator : fitted estimator
            The model to evaluate
        X : DataFrame
            Features including 'TOTAL_OVER_UNDER_LINE'
        y_true : array-like
            Actual total points
        
        Returns:
        --------
        float : Betting accuracy score
        """
        y_pred = estimator.predict(X)
        betting_line = X['TOTAL_OVER_UNDER_LINE'].values
        return over_under_betting_accuracy(y_true, y_pred, betting_line)


# Create a scorer object
over_under_scorer = OverUnderScorer()

In [30]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

from tabpfn import TabPFNRegressor
from tabpfn.constants import ModelVersion


# Initialize the regressor
regressor = TabPFNRegressor(device = 'cuda')  # Uses TabPFN-2.5 weights, trained on synthetic data only.
# To use TabPFN v2:
# regressor = TabPFNRegressor.create_default_for_version(ModelVersion.V2)
regressor.fit(X_train, y_train)

# Predict on the test set
predictions = regressor.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, predictions)
r2 = r2_score(y_test, predictions)
mae = mean_absolute_error(y_test, predictions)
betting_accuracy = over_under_betting_accuracy(y_test, predictions, X_test['TOTAL_OVER_UNDER_LINE'].values)

print("Mean Squared Error (MSE):", mse)
print("R² Score:", r2)
print("Mean Absolute Error (MAE):", mae)
print("Over/Under Betting Accuracy:", betting_accuracy)

OutOfMemoryError: CUDA out of memory. Tried to allocate 6.49 GiB. GPU 0 has a total capacity of 5.92 GiB of which 5.39 GiB is free. Including non-PyTorch memory, this process has 390.00 MiB memory in use. Of the allocated memory 288.48 MiB is allocated by PyTorch, and 21.52 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)